# Задача классификации ресторанных отзывов
https://www.kaggle.com/himanshupoddar/zomato-bangalore-restaurants

## Цель: классифицировать отличить хорошие отзывы от плохих. Плохими будем считать те, чей рейтинг меньше или равен 3. Хорошими будем считать все остальные отзывы (рейтинг больше трёх)

In [1]:
import spacy
import pandas as pd
import sys
import csv
import re
from tqdm import tqdm
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

Небольшой хак для нормального чтения csv с большим количеством символов в строке

In [2]:
maxInt = sys.maxsize
while True:
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [3]:
data = pd.read_csv('C:\\NLP\\zomato-bangalore-restaurants\\zomato.csv', sep = ',', engine = 'python')
data[:10]

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari
5,https://www.zomato.com/bangalore/timepass-dinn...,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,Yes,No,3.8/5,286,+91 9980040002\r\n+91 9980063005,Basavanagudi,Casual Dining,"Onion Rings, Pasta, Kadhai Paneer, Salads, Sal...",North Indian,600,"[('Rated 3.0', 'RATED\n Food 3/5\nAmbience 3/...",[],Buffet,Banashankari
6,https://www.zomato.com/bangalore/rosewood-inte...,"19/1, New Timberyard Layout, Beside Satellite ...",Rosewood International Hotel - Bar & Restaurant,No,No,3.6/5,8,+91 9731716688\r\n080 26740366,Mysore Road,Casual Dining,NaN,"North Indian, South Indian, Andhra, Chinese",800,"[('Rated 5.0', 'RATED\n Awesome food ??Great ...",[],Buffet,Banashankari
7,https://www.zomato.com/bangalore/onesta-banash...,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,Yes,Yes,4.6/5,2556,080 48653961\r\n080 48655715,Banashankari,"Casual Dining, Cafe","Farmhouse Pizza, Chocolate Banana, Virgin Moji...","Pizza, Cafe, Italian",600,"[('Rated 5.0', 'RATED\n I personally really l...",[],Cafes,Banashankari
8,https://www.zomato.com/bangalore/penthouse-caf...,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,Yes,No,4.0/5,324,+91 8884135549\r\n+91 9449449316,Banashankari,Cafe,"Pizza, Mocktails, Coffee, Nachos, Salad, Pasta...","Cafe, Italian, Continental",700,"[('Rated 3.0', ""RATED\n I had been to this pl...",[],Cafes,Banashankari
9,https://www.zomato.com/bangalore/smacznego-ban...,"2470, 21 Main Road, 25th Cross, Banashankari, ...",Smacznego,Yes,No,4.2/5,504,+91 9945230807\r\n+91 9743804471,Banashankari,Cafe,"Waffles, Pasta, Coleslaw Sandwich, Choco Waffl...","Cafe, Mexican, Italian, Momos, Beverages",550,"[('Rated 4.0', ""RATED\n Easy to locate\nVFM 3...",[],Cafes,Banashankari


Нам не нужны никакие столбцы, кроме reviews_list

In [4]:
data = data['reviews_list']

Взглянем подробнее на какой-нибудь один пример из выборки целиком?

In [5]:
data[0]

'[(\'Rated 4.0\', \'RATED\\n  A beautiful place to dine in.The interiors take you back to the Mughal era. The lightings are just perfect.We went there on the occasion of Christmas and so they had only limited items available. But the taste and service was not compromised at all.The only complaint is that the breads could have been better.Would surely like to come here again.\'), (\'Rated 4.0\', \'RATED\\n  I was here for dinner with my family on a weekday. The restaurant was completely empty. Ambience is good with some good old hindi music. Seating arrangement are good too. We ordered masala papad, panner and baby corn starters, lemon and corrionder soup, butter roti, olive and chilli paratha. Food was fresh and good, service is good too. Good for family hangout.\\nCheers\'), (\'Rated 2.0\', \'RATED\\n  Its a restaurant near to Banashankari BDA. Me along with few of my office friends visited to have buffet but unfortunately they only provide veg buffet. On inquiring they said this plac

Что мы видим? Огромную строку. Будем обрабатывать данные. Уже явно проще, чем XML, да ещё и балансирование.

In [6]:
reviews = []

for ratings in tqdm(data):
    ratings = eval(ratings)
    for rate, review in ratings:
        if rate:
            rate = rate.strip("Rated").strip()
            review = review.strip('RATED').strip()
            rate = float(rate)
            reviews.append([rate, review])

100%|██████████████████████████████████████████████████████████████████████████| 51717/51717 [00:46<00:00, 1105.75it/s]


In [7]:
reviews[0]

[4.0,
 'A beautiful place to dine in.The interiors take you back to the Mughal era. The lightings are just perfect.We went there on the occasion of Christmas and so they had only limited items available. But the taste and service was not compromised at all.The only complaint is that the breads could have been better.Would surely like to come here again.']

In [8]:
len(reviews) 

1319968

In [9]:
data=pd.DataFrame(reviews,columns=['rating','review'])

In [10]:
data[:5]

,rating,review
0,4.0,A beautiful place to dine in.The interiors tak...
1,4.0,I was here for dinner with my family on a week...
2,2.0,Its a restaurant near to Banashankari BDA. Me ...
3,4.0,We went here on a weekend and one of us had th...
4,5.0,The best thing about the place is itГѓВѓГѓВ‚...


In [11]:
data['review'][0]

'A beautiful place to dine in.The interiors take you back to the Mughal era. The lightings are just perfect.We went there on the occasion of Christmas and so they had only limited items available. But the taste and service was not compromised at all.The only complaint is that the breads could have been better.Would surely like to come here again.'

Сперва стоит обработать рейтинги. Как мы помним, плохие отзывы <= 3, хорошие > 3

In [12]:
new_rate_col = data['rating'].apply(lambda x: 1 if int(x) > 3 else 0)

In [13]:
data['rating'] = new_rate_col

In [14]:
data[:5]

,rating,review
0,1,A beautiful place to dine in.The interiors tak...
1,1,I was here for dinner with my family on a week...
2,0,Its a restaurant near to Banashankari BDA. Me ...
3,1,We went here on a weekend and one of us had th...
4,1,The best thing about the place is itГѓВѓГѓВ‚...


In [15]:
rates = data['rating'].value_counts()
rates

1    833907
0    486061
Name: rating, dtype: int64

Ок, теперь есть смысл заняться обработкой рецензий
* Приведём текст к нижнему регистру
* Вырежем знаки препинания
* Вырежем спец. символы
* Вырежем стоп-слова
* Заменим can't на can not, don't на do not и пр. 

In [16]:
data['review'][7]

'You canГѓВѓ\x83ГѓВ‚\x83ГѓВѓ\x82ГѓВ‚\x82ГѓВѓ\x83ГѓВ‚\x82ГѓВѓ\x82ГѓВ‚\x92t go wrong with Jalsa. Never been a fan of their buffet and thus always order alacarteГѓВѓ\x83ГѓВ‚\x83ГѓВѓ\x82ГѓВ‚\x82ГѓВѓ\x83ГѓВ‚\x82ГѓВѓ\x82ГѓВ‚\x92. Service at times can be on the slower side but food is worth the wait.'

In [17]:
replacements = {'cant': 'can not', 'arent': 'are not', 'isnt': 'is not', 'its': 'it is', 'wont': 'will not', 'didnt' : 'did not', 'wasnt' : 'was not', 'dont': 'do not', 'werent': 'were not'}

In [18]:
data['review'][7]

'You canГѓВѓ\x83ГѓВ‚\x83ГѓВѓ\x82ГѓВ‚\x82ГѓВѓ\x83ГѓВ‚\x82ГѓВѓ\x82ГѓВ‚\x92t go wrong with Jalsa. Never been a fan of their buffet and thus always order alacarteГѓВѓ\x83ГѓВ‚\x83ГѓВѓ\x82ГѓВ‚\x82ГѓВѓ\x83ГѓВ‚\x82ГѓВѓ\x82ГѓВ‚\x92. Service at times can be on the slower side but food is worth the wait.'

In [19]:
data['review'] = data['review'].apply(lambda x : x.replace('\n', ' '))
data['review'] = data['review'].apply(lambda x : re.sub('[^a-zA-Z0-9\s]',"",x))
for key, value in replacements.items():
    data['review'] = data['review'].apply(lambda x : x.replace(key, value))


In [20]:
data['review'][7]

'You can not go wrong with Jalsa Never been a fan of their buffet and thus always order alacarte Service at times can be on the slower side but food is worth the wait'

Предобработали. Теперь надо разделить выборку на тестовую и тренировочную

In [21]:
len(data) 

1319968

In [22]:
data = data[:100000]
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['rating'], test_size = 0.3, 
                                                    random_state = 7)

Нижний регистр и стоп-слова, да. Работаем с spaCy

In [23]:

X_train = [s.lower() for s in X_train]
X_test = [s.lower() for s in X_test]

In [24]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

punctuations = string.punctuation

nlp = spacy.load('en_core_web_md')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

parser = English()

def spacy_tokenizer(sentence):
    mytokens = parser(sentence)

    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    return mytokens

In [25]:
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}


def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [26]:

tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [27]:

classifier = LogisticRegression(max_iter=4000)
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x000001ED61B59CC8>),
                ('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop...
                                 tokenizer=<function spacy_tokenizer at 0x000001ED0E9731F8>,
                                 use_idf=True, vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
         

In [28]:
predicted = pipe.predict(X_train)
print("Train accuracy:",metrics.accuracy_score(y_train, predicted))

Train accuracy: 0.9179857142857143


In [29]:
predicted = pipe.predict(X_test)
print("Test accuracy:",metrics.accuracy_score(y_test, predicted))

Test accuracy: 0.9049333333333334


In [30]:
X_train = [ re.sub('[0-9]*',"",x) for x in X_train]
X_test = [ re.sub('[0-9]*',"",x) for x in X_test]
#X_test = X_test.apply(lambda x : re.sub('[0-9]*',"",x))

In [31]:
X_train[1]

'the place is nicely set the menu looks promising but the quantity is so disappointing even little elves would need  servings the pricing is equal to it is competition but the quantity is just ridiculous the taste is usual nothing authentic i just do not understand the pricing id definitely not recommend it  pricing   ambience   quantity   quality  '

Можно попробовать взять эмбеддинги из spaCy и посмотреть что получится(вряд ли что-то хорошее)

In [32]:
X_train_tokenized = [nlp(sample).vector for sample in X_train]
X_test_tokenized = [nlp(sample).vector  for sample in X_test]

In [33]:
classifier = LogisticRegression(max_iter=4000)
classifier.fit(X_train_tokenized, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=4000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
predicted = classifier.predict(X_train_tokenized)
print("Train accuracy:",metrics.accuracy_score(y_train, predicted))

Train accuracy: 0.8516


In [36]:
predicted = classifier.predict(X_test_tokenized)
print("Test accuracy:",metrics.accuracy_score(y_test, predicted))

Test accuracy: 0.8507
